**Final Project: Milestone #1**

by Daniel Alpert, Brendan Kent, Thomas Negron

In [45]:
# import libraries
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests
import urllib2
from sklearn.linear_model import LinearRegression as Lin_Reg
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
import sklearn.preprocessing as Preprocessing
import itertools as it
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy as sp
from itertools import combinations
import unicodedata
import subprocess
import time
from collections import namedtuple
%matplotlib inline

*API code to access the genre and movie poster path of your favorite movie:*

TMDb API username: brendankent

TMDb API password: stat121b

TMDb API Key: d9913957aa0163f103745e9444065812

In [2]:
# read in json object with movie info for "Blades of Glory"
url = "https://api.themoviedb.org/3/search/movie?api_key=d9913957aa0163f103745e9444065812&query=Blades+of+Glory"
blades_of_glory = json.load(urllib2.urlopen(url))

In [3]:
url_dhead = 'https://api.themoviedb.org/3/discover/movie?api_key=d9913957aa0163f103745e9444065812'

In [4]:
# create lists for genres corresponding to TIMDb genre ids
url = "https://api.themoviedb.org/3/genre/movie/list?api_key=d9913957aa0163f103745e9444065812&language=en-US"
genre_ids_json = json.load(urllib2.urlopen(url))
ids = []
genres = []

for i in genre_ids_json['genres']:
    
    ids.append(i['id'])
    genres.append(str(i['name']))

In [5]:
# get genre names for "Blades of Glory" from genre ids
bog_tmdb_genres = []

for i in blades_of_glory['results'][0]['genre_ids']:

    bog_tmdb_genres.append(genres[ids.index(i)])
    
print "Blades of Glory genres:", bog_tmdb_genres

Blades of Glory genres: ['Action', 'Comedy', 'Drama']


In [6]:
# get movie poster path for "Blades of Glory"
image_path = 'https://image.tmdb.org/t/p/w500/' + blades_of_glory['results'][0]['poster_path']
print 'Blades of Glory poster path:', image_path

Blades of Glory poster path: https://image.tmdb.org/t/p/w500//7vbtLFfN00BaczHjOaPWui4aqg1.jpg


*Genre for this movie listed by TMDb and IMDb:*

In [7]:
# print "Blades of Glory" TMDb genres
print "Blades of Glory TMDb genres:", bog_tmdb_genres

Blades of Glory TMDb genres: ['Action', 'Comedy', 'Drama']


In [78]:
# Lists for storing data
adult = []
genre_ids = []
ids = []
original_language = []
original_title = []
overview = []
popularity = []
poster_path = []
release_date = []
title = []
video = []
vote_average = []
vote_count = []

for yr in range(1987,2017):
    # Can only make 40 calls to the site every 10 seconds
    time.sleep(5)
    for pg in range(1,21):
        # Print year and page to make sure we are getting all the data we intended 
        print yr, pg
        url_loop = url_dhead+'&primary_release_year='+str(yr)+'&sort_by=popularity.desc&page='+str(pg)
        result_loop = json.load(urllib2.urlopen(url_loop))['results']
        
        for movie in result_loop:
        
            adult.append(movie['adult'])
            genre_ids.append(movie['genre_ids'])
            ids.append(movie['id'])
            original_language.append(movie['original_language'])
            original_title.append(movie['original_title'].encode('ascii', 'ignore').decode('ascii'))
            overview.append(movie['overview'].encode('ascii', 'ignore').decode('ascii'))
            popularity.append(movie['popularity'])
            poster_path.append(movie['poster_path'])
            release_date.append(movie['release_date'])
            title.append(movie['title'].encode('ascii', 'ignore').decode('ascii'))
            video.append(movie['video'])
            vote_average.append(movie['vote_average'])
            vote_count.append(movie['vote_count'])
        

1987 1
1987 2
1987 3
1987 4
1987 5
1987 6
1987 7
1987 8
1987 9
1987 10
1987 11
1987 12
1987 13
1987 14
1987 15
1987 16
1987 17
1987 18
1987 19
1987 20
1988 1
1988 2
1988 3
1988 4
1988 5
1988 6
1988 7
1988 8
1988 9
1988 10
1988 11
1988 12
1988 13
1988 14
1988 15
1988 16
1988 17
1988 18
1988 19
1988 20
1989 1
1989 2
1989 3
1989 4
1989 5
1989 6
1989 7
1989 8
1989 9
1989 10
1989 11
1989 12
1989 13
1989 14
1989 15
1989 16
1989 17
1989 18
1989 19
1989 20
1990 1
1990 2
1990 3
1990 4
1990 5
1990 6
1990 7
1990 8
1990 9
1990 10
1990 11
1990 12
1990 13
1990 14
1990 15
1990 16
1990 17
1990 18
1990 19
1990 20
1991 1
1991 2
1991 3
1991 4
1991 5
1991 6
1991 7
1991 8
1991 9
1991 10
1991 11
1991 12
1991 13
1991 14
1991 15
1991 16
1991 17
1991 18
1991 19
1991 20
1992 1
1992 2
1992 3
1992 4
1992 5
1992 6
1992 7
1992 8
1992 9
1992 10
1992 11
1992 12
1992 13
1992 14
1992 15
1992 16
1992 17
1992 18
1992 19
1992 20
1993 1
1993 2
1993 3
1993 4
1993 5
1993 6
1993 7
1993 8
1993 9
1993 10
1993 11
1993 12
1993 13

In [36]:
# Should have 30 years x 20 pages x 20 movies = 12000
print len(title)

12000


In [79]:
df = pd.DataFrame(data = {'adult':adult, 
                          'genre_ids':genre_ids, 
                          'id':ids, 
                          'orig_lang':original_language,
                          'orig_title':original_title, 
                          'overview':overview, 
                          'popularity':popularity,
                          'poster_path':poster_path,
                          'release_date':release_date, 
                          'title':title, 
                          'video':video,
                          'vote_average':vote_average, 
                          'vote_count':vote_count
                         })

In [80]:
df.to_csv('movie_df.csv')

In [81]:
movie

{u'adult': False,
 u'backdrop_path': u'/c6gLbAnjxclnHizyUBN2841jFPH.jpg',
 u'genre_ids': [18, 10749],
 u'id': 352890,
 u'original_language': u'en',
 u'original_title': u'Morris from America',
 u'overview': u'The romantic and coming-of-age misadventures of a 13-year-old American living in Germany.',
 u'popularity': 1.547952,
 u'poster_path': u'/x0z4SHamnyV7QpFHMtEWa58FEE4.jpg',
 u'release_date': u'2016-08-19',
 u'title': u'Morris from America',
 u'video': False,
 u'vote_average': 5.4,
 u'vote_count': 24}

*A list of the 10 most popular movies of 2016 from TMDb and their genre obtained via the API:*

In [12]:
url = "https://api.themoviedb.org/3/discover/movie?api_key=d9913957aa0163f103745e9444065812&sort_by=popularity.desc&primary_release_year=201"
pop = json.load(urllib2.urlopen(url))

In [14]:
for i in range(1, 10):
    
    movie = pop['results'][i]
    
    print movie['original_title'], movie['']

Fantastic Beasts and Where to Find Them
Finding Dory
Deadpool
Rogue One: A Star Wars Story
Arrival
Doctor Strange
Captain America: Civil War
Underworld: Blood Wars
Zootopia


In [20]:
url = "https://api.themoviedb.org/3/discover/movie?api_key=d9913957aa0163f103745e9444065812&primary_release_year=201"
year = json.load(urllib2.urlopen(url))

In [21]:
len(year['results'])

20